In [19]:
import pickle
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime
import glob

def sliding_window(data,stepSize,windowSize):
    for x in range(0,len(data)-windowSize+1,stepSize):
        yield(x,data[x:x+windowSize])

def pred_window(clf,data):
    test_set=[]
    for(x,window) in sliding_window(data,1,6): 
        wind=window.reshape(180)
        test_set.append(wind)
    print(np.shape(test_set))
    return clf.predict(test_set)

filename='./featuredata/compfeature.csv'
file=open(filename, newline='')
reader=csv.reader(file)
#context=reader[seq(1, nrow(reader), 2),]
#data = reader[seq(1, nrow(reader), 2),]
i=0
data=[]

In [20]:
# load svm
filename='./svm.sav'
svm = pickle.load(open(filename, 'rb'))


In [21]:
# load knn
filename='./knn.sav'
knn = pickle.load(open(filename, 'rb'))


In [22]:
# load rf
filename='./rf.sav'
rf = pickle.load(open(filename, 'rb'))


In [23]:
for row in reader:
    i+=1
    if(int(i)%2==1):
        context=row
    else:
        array=np.array(row)
        array=array.reshape(600,6,5)
        data.append(array)

print((data))
print(len(array))


[array([[['0.109208256', '-1.7843749599999998', '9.0461144',
         '-46.062799999999996', '0.7251999999999998'],
        ['-41.958', '0.4620195313690096', '4.081836462739785',
         '0.7295386787283044', '55.073357516679515'],
        ['9.928771573563367', '33.609209571187485', '-0.0119641',
         '-3.5342', '8.92284'],
        ['-29.12', '-1.12', '-37.87', '-0.167498', '-5.74038'],
        ['8.42034', '-111.23', '-3.64', '-60.34', '0.425922'],
        ['2.37129', '9.73161', '-9.24', '5.67', '-16.87']],

       [['0.9073580399999999', '-5.1460992', '8.4712616',
         '0.08399999999999994', '0.053200000000000004'],
        ['0.29400000000000004', '0.10040680641808301',
         '0.27717151241669846', '0.07809201289146017',
         '6.145792707210356'],
        ['1.7234540202744024', '2.8082478523093366', '0.911665',
         '-5.1661', '8.49931'],
        ['1.4', '-0.14', '0.14', '0.863809', '-5.29532'],
        ['8.42034', '-1.75', '-0.84', '-0.91', '0.937986'],
        ['

In [24]:
for x in data:
    print(pred_window(svm,x))

(595, 180)
[ 0 14 14 14 14 14 14 14 14  0  0  0  0  0  0 14 14 14 14 14 14 14 14  3
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14  0  0  0  0  0  0  0 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14  0  0  0  0  0  0  0  0  0  0  0 14 14 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14  0 14 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14 14  5  5  0  5 14  3 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14  0  0  0  0  0  0  0
  0  0  0  0  0  0 14 14 14 14 14 14 14 14  1  0  5  5  0 14 14  1 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 14  0 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 

In [25]:
for x in data:
    print(pred_window(knn,x))



TypeError: pred_window() missing 1 required positional argument: 'data'

In [26]:
for x in data:
    print(pred_window(rf,x))


(595, 180)
[ 8  1  2  1  1  1  1  1  1  1  9  8  0  6  1  8  1 12 12  2  2  6 12  8
  1  1  1  2  1 14  2  6  2  8  2  8  8  8  0  1  2  8  1  1  2  2  2  2
  2  2  2  2  2  2  8 12  1 12 12  1  1  2  2  0  6  1  8  8  1  8  9  2
  0  9  2  0  8  1  2  2  1  2  0  8  9 12  0  9 12 12  1  1  1  1  1  1
  1  1  1  1  1  1  2  1  1  1  8  8  1  2  1 14 12  8  1  1  1  1  8  1
  1  1  1  1  1  1  2  1  2  1  1 12 12  1  1  8  1  1  1  1  2  1  1  1
  1  1  1  1  1  1  1  1  1  9  8  1  6  6  0  1  1  8  2 12  1  1  2  2
  8  1  1  2  2  2  8 12 12 12 12  1  0  9 12 12 14  2  0  0  0 12  8  0
  0 12 14 14 12  1  1  1  1  1  8  1  1  1  1  1 14  8  8  1  1  1  1  1
  1  1  1  1  1  1  2  1  2  1  1  1  1  1  0  9  1  0  1  2  1  2  1  0
  8  1  0 12  1  0  9  0  8  0  9 12  2  1 12 14  0  8  1  1  8  1  1  0
  0  1  1  1  1  1 12 12  1  1  9  1  0  1  1  1  6  1  1  2  1  2  1  2
  2  1  1  0  0  8  1 12 12 12  6 14  8  8  9 12  1  0  9  8  8  8  8  8
  2  2  8  2  2  1  1  9 12 12 12  8  1 